In [ ]:
# dependencies import
from common_dependencies import *
import logging
logger = logging.getLogger('main.(02-05)v_test.ipynb')

In [ ]:
# paths for files with original data
PATH_TO_DATA = {
    'run_1': 
        ('data/original_data/run_1/run1_WM32_data.csv',
        'data/original_data/run_1/run1_WM32_defects.csv',
        'data/original_data/run_1/run1_WM32_pipe.csv'),
    'run_2':
        ('data/original_data/run_2/run2_WM32_data.csv',
        'data/original_data/run_2/run2_WM32_defects.csv',
        'data/original_data/run_2/run2_WM32_pipe.csv')
}

PATH_TO_MODEL = 'networks/CNN'

CROP_SIZE = 16
CROP_STEP = 16
RUN = 2
MODEL_VER = '04'
MODEL_NUM = '03'
XSHIFT = 200

for name in os.listdir(PATH_TO_MODEL):
    res = re.match(F'(id=v{MODEL_VER}n{MODEL_NUM}).*', name)
    if not res is None:
        PATH_TO_MODEL += '/' + res[0]
        break

# детерминация случайных величин, отвечающих за выбор первоначальных весов и биасов
tf.compat.v1.set_random_seed(290)
tf.random.set_seed(290)

In [ ]:
x_df, y_df = dw.get_x_and_y_data(*PATH_TO_DATA[f'run_{RUN}'])

In [ ]:
dw.draw_defects_map(y_df, title='Считанная развернутая карта дефектов')
#dw.draw_zeros_quantity_in_data_df(x_df)

In [ ]:
x_df = dw.extend_df_for_crops_dividing(x_df, crop_size=CROP_SIZE, crop_step=CROP_STEP)
y_df = dw.extend_df_for_crops_dividing(y_df, crop_size=CROP_SIZE, crop_step=CROP_STEP)
dw.draw_defects_map(y_df, title='Расширенная развернутая карта дефектов')

In [ ]:
# преобразовать данные к виду (измерение, размер х, размер у, каналы)
# для x_data_time каналы = 32
# для x_data_amp каналы = 32
# для y_data каналы = 1
(x_data_time,
x_data_amp) = dw.reshape_x_df_to_image_like_numpy(x_df, CROP_SIZE, CROP_STEP)
y_data = dw.reshape_y_df_to_image_like_numpy(y_df, CROP_SIZE, CROP_STEP)

In [ ]:
# нормализовать входные данные
x_data_time = dw.standardize_data(x_data_time)
x_data_amp = dw.standardize_data(x_data_amp)

In [ ]:
# Аугментировать данные
#x_data_time = dp.augment_data(x_data_time) 
#x_data_amp = dp.augment_data(x_data_amp) 
#y_data = dp.augment_data(y_data) 

In [ ]:
# создание выходных данных бинарного вида (одномерный массив типа bool)
y_data = dw.create_binary_arr_from_mask_arr(y_data)

In [ ]:
logger.debug(f'{x_data_time.shape=}')
logger.debug(f'{x_data_amp.shape=}')
logger.debug(f'{y_data.shape=}\n')

In [ ]:
# загрузка модели
model = keras.models.load_model(PATH_TO_MODEL)

In [ ]:
# тест модели
res = model.evaluate([x_data_time, x_data_amp], y_data, batch_size = 32)

In [ ]:
logger.debug(res)